<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex10/larissa_santesso/Aula_10_Exerc%C3%ADcio_Larissa_Santesso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
nome = "Larissa Antonelli Santesso"
print(f'Meu nome é {nome}')

Meu nome é Larissa Antonelli Santesso


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [ ]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 59.3 MB/s 
     |████████████████████████████████| 6.6 MB 60.2 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [ ]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [ ]:
# Check which GPU we are using
!nvidia-smi

Wed Jun  8 17:43:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

Mais sobre` batch_encode_plus`: https://huggingface.co/docs/transformers/internal/tokenization_utils#transformers.tokenization_utils_base.PreTrainedTokenizerBase.batch_encode_plus

In [ ]:
from typing import List


def tokenize(text: str, tokenizer):
    # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
    return tokenizer.batch_encode_plus(text, return_tensors='pt', add_special_tokens= False,  padding="longest").input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int, folder_name: str, data_type="train", iter_texts=1000, i_init=0):
        path = "/content/gdrive/MyDrive/Colab Notebooks/modelos_Aula09/"+folder_name
        self.tokens_ids =[]
        for i in range(i_init,len(texts), iter_texts):
            # Ideia de salvar os dados para aproveitar melhor a RAM baseada no exercício da Aula 08 do aluno Patrick Ferreira
            try:
                self.load_x = np.load(path+"x_"+ str(i) + "_"+ str(i+iter_texts) +"_" + data_type + ".npy", mmap_mode="r", allow_pickle=True)
                self.tokens_ids.append(torch.tensor(self.load_x))
                print(str(i) + " to "+ str(i+iter_texts)+" lines loaded")

            except Exception as e:
                output_tokenize = tokenize(texts[i:i+iter_texts], tokenizer)
                output_tokenize = torch.nn.functional.pad(output_tokenize, (0,max((max_seq_length-int(output_tokenize.shape[1]),1))))
                self.tokens_ids =[]
                shape_iter = output_tokenize.shape[1]
                for j in range(0,shape_iter-1, max_seq_length-1):    # Ideia do slicing baseada no notebook do Pedro Gengo   
                    if (j + max_seq_length) < int(shape_iter):
                        batch_seq = output_tokenize[:,j:j+max_seq_length]

                    else:
                        batch_seq = output_tokenize[:,-max_seq_length:]
                    
                    batch_seq = batch_seq[torch.sum(batch_seq, dim=1)!=0]
                    self.tokens_ids.extend(torch.cat([torch.tensor(tokenizer.cls_token_id).long().repeat(batch_seq.shape[0])[:, None],batch_seq], axis=1))
                
                self.tokens_ids = torch.stack(self.tokens_ids)
                
                print(f"Saving: {i} to {i+iter_texts} lines - shape ={self.tokens_ids.shape}")
                np.save(path+"x_"+ str(i) + "_"+ str(i+iter_texts) + "_" + data_type + ".npy", np.array(self.tokens_ids))

        try:
            self.tokens_ids = torch.vstack(self.tokens_ids)
        
        except:
            print("Don't need to stack tensor")

    def __len__(self):

        return len(self.tokens_ids)

    def __getitem__(self, idx):
        return self.tokens_ids[idx,:-1].long(), self.tokens_ids[idx,1:].long()

## Testando se a implementação do MyDataset está correta

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 2
print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert de dataset.')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Saving: 0 to 1000 lines - shape =torch.Size([2, 10])
Don't need to stack tensor
Passou no assert de tamanho do dataset.
Passou no assert de dataset.


In [ ]:
len(dummy_dataset)

2

In [ ]:
first_batch_input, correct_first_batch_input

(tensor([[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
         [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]]),
 tensor([[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
         [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]]))

In [ ]:
first_batch_target, correct_first_batch_target

(tensor([[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
         [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]]),
 tensor([[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
         [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]]))

In [ ]:
dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=3,folder_name="dataset_04/")
dummy_loader = DataLoader(dummy_dataset, batch_size=1, shuffle=False)

next(iter(dummy_loader))

Saving: 0 to 1000 lines - shape =torch.Size([6, 4])
Don't need to stack tensor


[tensor([[  101,  3396, 10303]]), tensor([[ 3396, 10303,   125]])]

In [ ]:
dummy_loader = DataLoader(dummy_dataset, batch_size=10, shuffle=False)

next(iter(dummy_loader))

[tensor([[  101,  3396, 10303],
         [  101,  1660,  5971],
         [  101,   125, 13239],
         [  101,   785,   125],
         [  101,  1847, 13779],
         [  101, 13779, 15616]]), tensor([[ 3396, 10303,   125],
         [ 1660,  5971,   785],
         [  125, 13239,     0],
         [  785,   125,  1847],
         [ 1847, 13779, 15616],
         [13779, 15616,     0]])]

# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [ ]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

--2022-06-08 21:40:45--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.214.128, 173.194.215.128, 173.194.216.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1230909256 (1.1G) [text/plain]
Saving to: ‘sample-1gb.txt’

sample-1gb.txt      100%[===================>]   1.15G  89.6MB/s    in 12s     

2022-06-08 21:40:57 (100 MB/s) - ‘sample-1gb.txt’ saved [1230909256/1230909256]



## Salvando o dataset em uma pasta da Drive

In [ ]:
# Load datasets
max_seq_length = 12

train_examples = 500
valid_examples = 100
test_examples = 100

texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

#max_lines = train_examples + valid_examples + test_examples
#print(f'Truncating to {max_lines} lines.')
#texts = texts[:max_lines]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

folder_save="dataset_04/"
training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length, folder_name=folder_save)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length, folder_name=folder_save, data_type="val", iter_texts=100)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length, folder_name=folder_save, data_type="test", iter_texts=100)

Read 250000 lines.
Saving: 0 to 1000 lines - shape =torch.Size([106473, 13])
Saving: 1000 to 2000 lines - shape =torch.Size([103069, 13])
Saving: 2000 to 3000 lines - shape =torch.Size([119189, 13])
Saving: 3000 to 4000 lines - shape =torch.Size([113051, 13])
Saving: 4000 to 5000 lines - shape =torch.Size([94192, 13])
Saving: 5000 to 6000 lines - shape =torch.Size([113454, 13])
Saving: 6000 to 7000 lines - shape =torch.Size([108846, 13])
Saving: 7000 to 8000 lines - shape =torch.Size([91483, 13])
Saving: 8000 to 9000 lines - shape =torch.Size([118739, 13])
Saving: 9000 to 10000 lines - shape =torch.Size([113324, 13])
Saving: 10000 to 11000 lines - shape =torch.Size([103453, 13])
Saving: 11000 to 12000 lines - shape =torch.Size([98252, 13])
Saving: 12000 to 13000 lines - shape =torch.Size([97600, 13])
Saving: 13000 to 14000 lines - shape =torch.Size([92441, 13])
Saving: 14000 to 15000 lines - shape =torch.Size([108823, 13])
Saving: 15000 to 16000 lines - shape =torch.Size([97246, 13])
S

## Carregando Dataset salvo na pasta do Drive

In [ ]:
# Load datasets
max_seq_length = 12

train_examples = 500
valid_examples = 100
test_examples = 100

texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

#max_lines = train_examples + valid_examples + test_examples
#print(f'Truncating to {max_lines} lines.')
#texts = texts[:max_lines]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

folder_save="dataset_04/"
training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length, folder_name=folder_save)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length, folder_name=folder_save, data_type="val", iter_texts=100)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length, folder_name=folder_save, data_type="test", iter_texts=100)

Read 250000 lines.
0 to 1000 lines loaded
1000 to 2000 lines loaded
2000 to 3000 lines loaded
3000 to 4000 lines loaded
4000 to 5000 lines loaded
5000 to 6000 lines loaded
6000 to 7000 lines loaded
7000 to 8000 lines loaded
8000 to 9000 lines loaded
9000 to 10000 lines loaded
10000 to 11000 lines loaded
11000 to 12000 lines loaded
12000 to 13000 lines loaded
13000 to 14000 lines loaded
14000 to 15000 lines loaded
15000 to 16000 lines loaded
16000 to 17000 lines loaded
17000 to 18000 lines loaded
18000 to 19000 lines loaded
19000 to 20000 lines loaded
20000 to 21000 lines loaded
21000 to 22000 lines loaded
22000 to 23000 lines loaded
23000 to 24000 lines loaded
24000 to 25000 lines loaded
25000 to 26000 lines loaded
26000 to 27000 lines loaded
27000 to 28000 lines loaded
28000 to 29000 lines loaded
29000 to 30000 lines loaded
30000 to 31000 lines loaded
31000 to 32000 lines loaded
32000 to 33000 lines loaded
33000 to 34000 lines loaded
34000 to 35000 lines loaded
35000 to 36000 lines lo

In [ ]:
training_dataset[0]

(tensor([  101, 20100,  2308,  3074,  1089,   481,   117,   146,  1189,   125,
         13254,   143]),
 tensor([20100,  2308,  3074,  1089,   481,   117,   146,  1189,   125, 13254,
           143,   122]))

In [ ]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 25515683
valid examples: 12081
test examples: 6803


# Arquitetura do modelo

In [ ]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, dim: int, n_heads: int):
        """
        Implements the Multi-Head Self-attention."

        Args:on.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_heads (int): number of heads.
            mask(bool): if applies mask or not
        """
        super(MultiHeadAttention, self).__init__()
        
        self.dim = dim
        self.n_heads = n_heads

        self.W_q = nn.Linear(dim, dim, bias = False)    # shape = (D, D)
        self.W_k = nn.Linear(dim, dim, bias = False)    # shape = (D, D)   
        self.W_v = nn.Linear(dim, dim, bias = False)    # shape = (D, D)
        self.W_o = nn.Linear(dim, dim, bias = False)    # shape = (D, D)

    def attention(self, q, k, v, mask=None):
        scores = torch.bmm(q, k.transpose(1,2)) #shape: (B*H, L, D/H) * (B*H, D/H, L) = (B*H, L, L)
        scores = scores/math.sqrt(self.dim) # scale by 1/sqrt(D)

        if mask is not None:
            # retornando escores para o shape (B, H, L, L) para aplicar a mascara
            scores = scores.view(self.batch_size, self.n_heads, self.context_size, self.context_size).masked_fill(mask == 0, float('-inf'))
            scores = scores.view(self.batch_size*self.n_heads, self.context_size, self.context_size)

        # retornando escores para o shape (B*H, L, L) para seguir os cálculos
        
        probs = torch.nn.functional.softmax(scores, dim=-1) # shape:   (B*H, L, L)
        out = torch.bmm(probs, v).view(self.batch_size, self.n_heads, self.context_size, int(self.dim/self.n_heads)) # shape:   (B, H, L, D/H)

        return out

    def forward(self, inputs, mask):
        self.batch_size = inputs.shape[0]   # shape = B
        self.context_size = inputs.shape[1] # shape = L

        q = self.W_q(inputs).reshape(self.batch_size, self.context_size, self.n_heads, self.dim//self.n_heads)  # shape = (B, L, H, D/H)
        k = self.W_k(inputs).reshape(self.batch_size, self.context_size, self.n_heads, self.dim//self.n_heads)  # shape = (B, L, H, D/H)
        v = self.W_v(inputs).reshape(self.batch_size, self.context_size, self.n_heads, self.dim//self.n_heads)  # shape = (B, L, H, D/H)

        # Changing shapes for: (B, H, L, D/H)
        q = q.transpose(1,2).contiguous().view(int(self.batch_size*self.n_heads), self.context_size, self.dim//self.n_heads) # shape = (B*H, L, D/H) 
        k = k.transpose(1,2).contiguous().view(int(self.batch_size*self.n_heads), self.context_size, self.dim//self.n_heads) # shape = (B*H, L, D/H) 
        v = v.transpose(1,2).contiguous().view(int(self.batch_size*self.n_heads), self.context_size, self.dim//self.n_heads) # shape = (B*H, L, D/H) 

        E = self.attention(q, k, v, mask)  # shape = (B, H, L, D/H)
        E = E.transpose(1,2).contiguous() # shape = (B, L, H, D/H)
        
        E = E.reshape(self.batch_size, self.context_size, self.dim) # shape = (B, L, D)

        E = self.W_o(E)  # shape = (B, L, D)

        return E

In [ ]:
class FeedForward(torch.nn.Module):
    def __init__(self, dim: int, hid_dim: int):
        """
        Implements the Multi-Head Self-attention."

        Args:on.
            dim (int): Dimension of the embedding layer for each word in the context.
            hid_dim (int): dimension of the hidden layer
        """
        super(FeedForward, self).__init__()
        
        self.dim = dim
        self.hid_dim = hid_dim

        self.linear1 = nn.Linear(self.dim, hid_dim)
        self.relu =  nn.ReLU()
        self.linear2 =  nn.Linear(hid_dim, self.dim)
        self.drop =  nn.Dropout(p=0.1)
        


    def forward(self, inputs):
        out = self.relu(self.linear1(inputs))
        out = self.drop(self.linear2(out))

        return out

In [ ]:
class LayerDecoder(torch.nn.Module):
    def __init__(self, dim: int, hid_dim: int, n_heads: int):
        """
        Implements the Multi-Head Self-attention."

        Args:on.
            dim (int): Dimension of the embedding layer for each word in the context.
            hid_dim (int): dimension of the hidden layer
            drop_rate (float): rate of the dropout to apply
        """
        super(LayerDecoder, self).__init__()
        
        self.dim = dim
        self.hid_dim = hid_dim
        self.n_heads = n_heads


        self.multihead_attention = MultiHeadAttention(self.dim, self.n_heads)
        self.feedforward = FeedForward(self.dim, self.hid_dim)
        self.drop =  nn.Dropout(p=0.1)

        # Layer Norms
        self.layer_norm1 = nn.LayerNorm(self.dim)
        self.layer_norm2 = nn.LayerNorm(self.dim)

    def forward(self, inputs, mask=None):

        # Multi Head Attention Block
        out = self.multihead_attention(inputs, mask)
        out = self.drop(out)
        out = self.layer_norm1(out+inputs)

        # Feed Forward Block
        out_ff = self.feedforward(out)
        out_ff = self.layer_norm2(out_ff+out)

        return out_ff

In [ ]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
            pad_token_id (int): id of the pad token that will be ignored in the attention.
        """
        super(LanguageModel, self).__init__()

        self.context_size = max_seq_length
        self.dim = dim
        self.n_heads = 4
        self.hidden_dim = 2*self.dim
        self.pad_token_id = pad_token_id

        # Embedding of the words
        self.embeddings_C = nn.Embedding(vocab_size, self.dim, padding_idx=self.pad_token_id)

        # Embedding of the words positions
        self.embeddings_P = nn.Embedding(self.context_size, self.dim)

        # Causal Mask
        self.causal_mask = torch.tril(torch.ones((self.context_size, self.context_size))).bool().to(device) # shape: (L, L)
        
        # Applying n_layers of the DecoderLayer
        self.layers_decoder = nn.ModuleList([
            LayerDecoder(self.dim, self.hidden_dim, self.n_heads)
            for _ in range(n_layers)])

        # Dropout
        self.dropout = nn.Dropout(p=0.1)

        # Output layer
        self.dense = nn.Linear(self.dim, vocab_size, bias = False)


    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)
            
        Returns:
            logits of shape (batch_size, max_seq_length, vocab_size)
        """
        embeds = self.embeddings_C(inputs) # embeds shape: (B, L, D)
        # self.embeddings_P.weight shape: (L, D)

        X = embeds + self.embeddings_P.weight # X shape: (B, L, D) 
        X = self.dropout(X)

        pad_mask = (inputs != self.pad_token_id).unsqueeze(1).unsqueeze(2) # shape: (B, 1, 1,  L)
        
        mask_padc = pad_mask & self.causal_mask  # (B, 1, L, L)

        for layer in self.layers_decoder:
            # X shape: (B, L, D)
            X = layer(X, mask = mask_padc)
        
        logits = self.dense(X) # logits shape: (B, L, V)

        return logits

## Exemplo da criação da máscara de padding e "no peak":

In [ ]:
t1 = torch.tril(torch.ones((9, 9))).bool()
t1

tensor([[ True, False, False, False, False, False, False, False, False],
        [ True,  True, False, False, False, False, False, False, False],
        [ True,  True,  True, False, False, False, False, False, False],
        [ True,  True,  True,  True, False, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False, False],
        [ True,  True,  True,  True,  True,  True, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True]])

In [ ]:
t2 = torch.tensor([[True, True, True, True, True, True, False, False, False],
                  [True, True, True, True, True, True, True, True, True],
                  [True, True, True, True, True, True, True, True, False], 
                  [True, True, True, True, True, True, True, False, False],
                  [True, True, True, True, False, False,False, False, False]]).unsqueeze(1).unsqueeze(2)
t2

tensor([[[[ True,  True,  True,  True,  True,  True, False, False, False]]],


        [[[ True,  True,  True,  True,  True,  True,  True,  True,  True]]],


        [[[ True,  True,  True,  True,  True,  True,  True,  True, False]]],


        [[[ True,  True,  True,  True,  True,  True,  True, False, False]]],


        [[[ True,  True,  True,  True, False, False, False, False, False]]]])

In [ ]:
t1.shape

torch.Size([9, 9])

In [ ]:
t2.shape

torch.Size([5, 1, 1, 9])

In [ ]:
t1_t2 = t1&t2

In [ ]:
(t1_t2).shape

torch.Size([5, 1, 9, 9])

## Teste o modelo com um exemplo

In [ ]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=512,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset, batch_size=5)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([5, 12])
sample_output.shape: torch.Size([5, 12, 29794])


In [ ]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 34716672


## Assert da Perplexidade


In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_len, vocab_size)
        target: a LongTensor of shape (batch_size, seq_len)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              34018
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [ ]:
max_examples = 150_000_000
eval_every_steps = 10000
lr = 3e-4
compare=float('inf')

model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=512,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            if valid_ppl<compare:
                compare=valid_ppl
                torch.save(model, "/content/gdrive/MyDrive/Colab Notebooks/modelos_Aula09/"+"model_v12.pt")
                with open("/content/gdrive/MyDrive/Colab Notebooks/modelos_Aula09/"+"valid_ppl_model_v12.txt", 'w') as f:
                    f.write("%s\n" % valid_ppl)
                f.close()
                
            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 34451.10, valid ppl: 26226.82
10000 steps; 10240000 examples so far; train ppl: 237.18, valid ppl: 180.02


In [ ]:
with open("/content/gdrive/MyDrive/Colab Notebooks/modelos_Aula09/"+"valid_ppl_model_v12.txt") as f:
    txt = list(f)
    compare = float(txt[-1])
    f.close()


In [ ]:
max_examples = 150_000_000
eval_every_steps = 10000
lr = 3e-4
print(compare)

model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=512,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)


model = torch.load("/content/gdrive/MyDrive/Colab Notebooks/modelos_Aula09/"+"model_v12.pt")
model.to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            if valid_ppl<compare:
                compare=valid_ppl
                torch.save(model, "/content/gdrive/MyDrive/Colab Notebooks/modelos_Aula09/"+"model_v12_run02.pt")
                with open("/content/gdrive/MyDrive/Colab Notebooks/modelos_Aula09/"+"valid_ppl_model_v12_run02.txt", 'w') as f:
                    f.write("%s\n" % valid_ppl)
                f.close()
                
            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

180.01938340763607
0 steps; 0 examples so far; train ppl: 176.08, valid ppl: 180.94
10000 steps; 10240000 examples so far; train ppl: 158.80, valid ppl: 160.14
20000 steps; 20480000 examples so far; train ppl: 148.55, valid ppl: 150.26
30000 steps; 30720000 examples so far; train ppl: 140.60, valid ppl: 144.65


In [ ]:
with open("/content/gdrive/MyDrive/Colab Notebooks/modelos_Aula09/"+"valid_ppl_model_v12_run02.txt") as f:
    txt = list(f)
    compare = float(txt[-1])
    f.close()


In [ ]:
max_examples = 150_000_000
eval_every_steps = 10000
lr = 3e-4
print(compare)

model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=512,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)


model = torch.load("/content/gdrive/MyDrive/Colab Notebooks/modelos_Aula09/"+"model_v12_run02.pt")
model.to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            if valid_ppl<compare:
                compare=valid_ppl
                torch.save(model, "/content/gdrive/MyDrive/Colab Notebooks/modelos_Aula09/"+"model_v12_run03.pt")
                with open("/content/gdrive/MyDrive/Colab Notebooks/modelos_Aula09/"+"valid_ppl_model_v12_run03.txt", 'w') as f:
                    f.write("%s\n" % valid_ppl)
                f.close()
                
            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

144.64617235144226
0 steps; 0 examples so far; train ppl: 141.90, valid ppl: 144.98
10000 steps; 10240000 examples so far; train ppl: 135.09, valid ppl: 141.29
20000 steps; 20480000 examples so far; train ppl: 132.56, valid ppl: 138.56


## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [ ]:
model = torch.load("/content/gdrive/MyDrive/Colab Notebooks/modelos_Aula09/"+"model_v12_run03.pt")
model.to(device)

LanguageModel(
  (embeddings_C): Embedding(29794, 512, padding_idx=0)
  (embeddings_P): Embedding(12, 512)
  (layers_decoder): ModuleList(
    (0): LayerDecoder(
      (multihead_attention): MultiHeadAttention(
        (W_q): Linear(in_features=512, out_features=512, bias=False)
        (W_k): Linear(in_features=512, out_features=512, bias=False)
        (W_v): Linear(in_features=512, out_features=512, bias=False)
        (W_o): Linear(in_features=512, out_features=512, bias=False)
      )
      (feedforward): FeedForward(
        (linear1): Linear(in_features=512, out_features=1024, bias=True)
        (relu): ReLU()
        (linear2): Linear(in_features=1024, out_features=512, bias=True)
        (drop): Dropout(p=0.1, inplace=False)
      )
      (drop): Dropout(p=0.1, inplace=False)
      (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (layer_norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
    (1): LayerDecoder(
      (multihead_attentio

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()
    
with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 112.10454591695101


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
def tokenize(text: str, tokenizer):

    return tokenizer.encode_plus(text, return_tensors=None, add_special_tokens= False).input_ids

In [ ]:
prompt = 'Eu gosto de comer muita pizza aos domingos pois me faz '
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer muita pizza aos domingos pois me faz a
Eu gosto de comer muita pizza aos domingos pois me faz a diferença
Eu gosto de comer muita pizza aos domingos pois me faz a diferença ser
Eu gosto de comer muita pizza aos domingos pois me faz a diferença ser mais
Eu gosto de comer muita pizza aos domingos pois me faz a diferença ser mais uma
Eu gosto de comer muita pizza aos domingos pois me faz a diferença ser mais uma coisa
Eu gosto de comer muita pizza aos domingos pois me faz a diferença ser mais uma coisa,
Eu gosto de comer muita pizza aos domingos pois me faz a diferença ser mais uma coisa, mas
Eu gosto de comer muita pizza aos domingos pois me faz a diferença ser mais uma coisa, mas não
Eu gosto de comer muita pizza aos domingos pois me faz a diferença ser mais uma coisa, mas não é
Eu gosto de comer muita pizza aos domingos pois me faz a diferença ser mais uma coisa, mas não é uma
Eu gosto de comer muita pizza aos domingos pois me faz a diferença ser mais uma coisa, mas n

In [ ]:
prompt = "Temos que pensar no futuro e guardar o que aprendemos na"
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Temos que pensar no futuro e guardar o que aprendemos na prática
Temos que pensar no futuro e guardar o que aprendemos na prática.
Temos que pensar no futuro e guardar o que aprendemos na prática. O
Temos que pensar no futuro e guardar o que aprendemos na prática. O que
Temos que pensar no futuro e guardar o que aprendemos na prática. O que é
Temos que pensar no futuro e guardar o que aprendemos na prática. O que é que
Temos que pensar no futuro e guardar o que aprendemos na prática. O que é que o
Temos que pensar no futuro e guardar o que aprendemos na prática. O que é que o homem
Temos que pensar no futuro e guardar o que aprendemos na prática. O que é que o homem não
Temos que pensar no futuro e guardar o que aprendemos na prática. O que é que o homem não tem
Temos que pensar no futuro e guardar o que aprendemos na prática. O que é que o homem não tem a
Temos que pensar no futuro e guardar o que aprendemos na prática. O que é que o homem não tem a ver
Temos que pensar no futuro e gu

In [ ]:
prompt = "A exposição dos quadros da Tarsila do Amaral ocorreu na"
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

A exposição dos quadros da Tarsila do Amaral ocorreu na sexta
A exposição dos quadros da Tarsila do Amaral ocorreu na sexta -
A exposição dos quadros da Tarsila do Amaral ocorreu na sexta - feira
A exposição dos quadros da Tarsila do Amaral ocorreu na sexta - feira,
A exposição dos quadros da Tarsila do Amaral ocorreu na sexta - feira, o
A exposição dos quadros da Tarsila do Amaral ocorreu na sexta - feira, o presidente
A exposição dos quadros da Tarsila do Amaral ocorreu na sexta - feira, o presidente da
A exposição dos quadros da Tarsila do Amaral ocorreu na sexta - feira, o presidente da Associação
A exposição dos quadros da Tarsila do Amaral ocorreu na sexta - feira, o presidente da Associação dos
A exposição dos quadros da Tarsila do Amaral ocorreu na sexta - feira, o presidente da Associação dos Mag
A exposição dos quadros da Tarsila do Amaral ocorreu na sexta - feira, o presidente da Associação dos Magist
A exposição dos quadros da Tarsila do Amaral ocorreu na sexta - feira, o p

## Aplicando modelo em textos do treino

In [ ]:
texts[1000]

'Palestra traz resultados de 10 anos de estudo sobre o tratamento da Esclerose Múltipla O Ciclo de Conferências do Instituto de Ciências Biológicas (ICB) Marc Jeannerod começa, nessa terça-feira, 6, trazendo como palestrantes, pesquisadores da UFJF cujos trabalhos se destacam pela excelência em suas respectivas áreas. Servindo como forma de divulgação e discussão do conhecimento, o evento — aberto a toda a comunidade acadêmica — oferece aos participantes uma oportunidade de se atualizarem em relação às principais pesquisas desenvolvidas na área. Com duração de 50 minutos, as conferências serão realizadas no Anfiteatro A do ICB, às 16h das primeiras terças-feiras de cada mês. Abrindo o Ciclo, a professora Ana Paula Ferreira irá tratar de seus trabalhos realizados na área da Imunologia, durante a palestra "Mecanismos imunoregulatórios envolvidos na Encelafalomielite Autoimune Experimental". Em sua fala, Ana Paula irá compartilhar os resultados obtidos ao longo de 10 anos de estudo sobre 

In [ ]:
prompt = "Palestra traz resultados de 10 anos de estudo sobre o tratamento"
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Palestra traz resultados de 10 anos de estudo sobre o tratamento de
Palestra traz resultados de 10 anos de estudo sobre o tratamento de um
Palestra traz resultados de 10 anos de estudo sobre o tratamento de um paciente
Palestra traz resultados de 10 anos de estudo sobre o tratamento de um paciente,
Palestra traz resultados de 10 anos de estudo sobre o tratamento de um paciente, que
Palestra traz resultados de 10 anos de estudo sobre o tratamento de um paciente, que é
Palestra traz resultados de 10 anos de estudo sobre o tratamento de um paciente, que é a
Palestra traz resultados de 10 anos de estudo sobre o tratamento de um paciente, que é a primeira
Palestra traz resultados de 10 anos de estudo sobre o tratamento de um paciente, que é a primeira vez
Palestra traz resultados de 10 anos de estudo sobre o tratamento de um paciente, que é a primeira vez que
Palestra traz resultados de 10 anos de estudo sobre o tratamento de um paciente, que é a primeira vez que o
Palestra traz resultados 

In [ ]:
texts[10000]

'"Escrito por professores e profissionais da área de Alimentação Coletiva, o livro aborda os desafios enfrentados por nutricionistas no dia a dia. Na obra, ressalta-se a importância da aplicação de conceitos teóricos à realidade prática de uma Unidade Produtora de Refeições, termo aplicado a serviços de alimentação externo ao domicílio.Levando em conta a crescente demanda por esse tipo de serviço no País, Administração de Unidades Produtoras de Refeições: Desafios e Perspectivas discorre sobre algumas perspectivas para a área, como a aplicação de novas tecnologias no setor e a importância da ergonomia para a saúde do manipulador de alimentos. Abrange, ainda, temas relacionados com administração e planejamento dessas unidades, aplicação da técnica dietética, gastronomia, ergonomia e segurança no trabalho, gestão de resíduos, controle de custos, treinamentos e consultoria na área de alimentação coletiva. Assim, esta publicação traz novas contribuições para acadêmicos e profissionais ao a

In [ ]:
prompt = "Escrito por professores e profissionais da área de Alimentação Coletiva, o livro aborda"
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Escrito por professores e profissionais da área de Alimentação Coletiva, o livro aborda a
Escrito por professores e profissionais da área de Alimentação Coletiva, o livro aborda a questão
Escrito por professores e profissionais da área de Alimentação Coletiva, o livro aborda a questão da
Escrito por professores e profissionais da área de Alimentação Coletiva, o livro aborda a questão da "
Escrito por professores e profissionais da área de Alimentação Coletiva, o livro aborda a questão da " "
Escrito por professores e profissionais da área de Alimentação Coletiva, o livro aborda a questão da " " História
Escrito por professores e profissionais da área de Alimentação Coletiva, o livro aborda a questão da " " História da
Escrito por professores e profissionais da área de Alimentação Coletiva, o livro aborda a questão da " " História da Vida
Escrito por professores e profissionais da área de Alimentação Coletiva, o livro aborda a questão da " " História da Vida "
Escrito por professores e 

In [ ]:
texts[490]

'Na estreia de Tite, o Brasil derrotou o Equador, por 3 a 0. A partida, válida pelas Eliminatórias da Copa do Mundo, foi disputada no início da noite desta quinta-feira, no Estádio Olímpico Atahualpa, em Quito. Os gols da Seleção Brasileira foram anotados por Neymar e Gabriel Jesus (2). A vitória conquistada fora de casa deixou o Brasil com 12 pontos, dentro da zona de classificação para o Mundial da Rússia de 2018. Na próxima terça-feira, dia 6, a Seleção Brasileira recebe a Colômbia na Arena da Amazônia, em Manaus (AM), às 21h45min. O jogo A etapa inicial da partida foi marcada por poucas chances de gol. O Equador começou melhor, mas o Brasil igualou as ações nos minutos seguintes. Aos 15, Gabriel Jesus chutou forte e mandou rente a trave. Já o time da casa, depois de uma saída de bola errada de Renato Augusto, ameaçou a meta defendida por Alisson, com Noboa, aos 35 minutos. No primeiro minuto do segundo tempo, Neymar chutou com perigo e mandou a bola perto da trave. Aos 24 minutos, 

In [ ]:
prompt = "Na estreia de Tite, o Brasil derrotou o Equador, por "
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Na estreia de Tite, o Brasil derrotou o Equador, por exemplo
Na estreia de Tite, o Brasil derrotou o Equador, por exemplo,
Na estreia de Tite, o Brasil derrotou o Equador, por exemplo, que
Na estreia de Tite, o Brasil derrotou o Equador, por exemplo, que o
Na estreia de Tite, o Brasil derrotou o Equador, por exemplo, que o Brasil
Na estreia de Tite, o Brasil derrotou o Equador, por exemplo, que o Brasil não
Na estreia de Tite, o Brasil derrotou o Equador, por exemplo, que o Brasil não tem
Na estreia de Tite, o Brasil derrotou o Equador, por exemplo, que o Brasil não tem o
Na estreia de Tite, o Brasil derrotou o Equador, por exemplo, que o Brasil não tem o direito
Na estreia de Tite, o Brasil derrotou o Equador, por exemplo, que o Brasil não tem o direito de
Na estreia de Tite, o Brasil derrotou o Equador, por exemplo, que o Brasil não tem o direito de ser
Na estreia de Tite, o Brasil derrotou o Equador, por exemplo, que o Brasil não tem o direito de ser um
Na estreia de Tite, o Brasil 

In [ ]:
prompt = "Na estreia de Tite o Brasil derrotou o Equador por 3"
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Na estreia de Tite o Brasil derrotou o Equador por 3 a
Na estreia de Tite o Brasil derrotou o Equador por 3 a 1
Na estreia de Tite o Brasil derrotou o Equador por 3 a 1.
Na estreia de Tite o Brasil derrotou o Equador por 3 a 1. 0
Na estreia de Tite o Brasil derrotou o Equador por 3 a 1. 0.
Na estreia de Tite o Brasil derrotou o Equador por 3 a 1. 0. O
Na estreia de Tite o Brasil derrotou o Equador por 3 a 1. 0. O time
Na estreia de Tite o Brasil derrotou o Equador por 3 a 1. 0. O time do
Na estreia de Tite o Brasil derrotou o Equador por 3 a 1. 0. O time do Grêmio
Na estreia de Tite o Brasil derrotou o Equador por 3 a 1. 0. O time do Grêmio,
Na estreia de Tite o Brasil derrotou o Equador por 3 a 1. 0. O time do Grêmio, que
Na estreia de Tite o Brasil derrotou o Equador por 3 a 1. 0. O time do Grêmio, que venceu
Na estreia de Tite o Brasil derrotou o Equador por 3 a 1. 0. O time do Grêmio, que venceu o
Na estreia de Tite o Brasil derrotou o Equador por 3 a 1. 0. O time do Grêmio, que ve

In [ ]:
prompt = "Na estreia de Tite, o Brasil derrotou o Equador por "
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Na estreia de Tite, o Brasil derrotou o Equador por 3
Na estreia de Tite, o Brasil derrotou o Equador por 3 a
Na estreia de Tite, o Brasil derrotou o Equador por 3 a 1
Na estreia de Tite, o Brasil derrotou o Equador por 3 a 1,
Na estreia de Tite, o Brasil derrotou o Equador por 3 a 1, em
Na estreia de Tite, o Brasil derrotou o Equador por 3 a 1, em São
Na estreia de Tite, o Brasil derrotou o Equador por 3 a 1, em São Paulo
Na estreia de Tite, o Brasil derrotou o Equador por 3 a 1, em São Paulo,
Na estreia de Tite, o Brasil derrotou o Equador por 3 a 1, em São Paulo, no
Na estreia de Tite, o Brasil derrotou o Equador por 3 a 1, em São Paulo, no Rio
Na estreia de Tite, o Brasil derrotou o Equador por 3 a 1, em São Paulo, no Rio de
Na estreia de Tite, o Brasil derrotou o Equador por 3 a 1, em São Paulo, no Rio de Janeiro
Na estreia de Tite, o Brasil derrotou o Equador por 3 a 1, em São Paulo, no Rio de Janeiro,
Na estreia de Tite, o Brasil derrotou o Equador por 3 a 1, em São Paulo, no Ri

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.